# Notebook for FlashMatching Analysis

## Imports & Setup

In [23]:
from __future__ import division
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter 
from root_pandas import read_root
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import cufflinks as cf
import random

#init_notebook_mode(connected=True)
#cf.go_offline()
%matplotlib inline

## Constants

In [2]:
nrPMT=32

filedict = {11: "Input/FitSimPhotons_electron.root",
            13: "Input/FitSimPhotons_muon.root",
            22: "Input/FitSimPhotons_gamma.root",
          2212: "Input/FitSimPhotons_proton.root"}

label = {   11: 'Electron',
            13: 'Muon',
            22: 'Gamma',
          2212: 'Proton'}

labels = {  11: 'Electrons',
            13: 'Muons',
            22: 'Gammas',
          2212: 'Protons'}

filedict = collections.OrderedDict(filedict)
label = collections.OrderedDict(label)
labels = collections.OrderedDict(labels)

ignorelists = ['event','subrun','run','flashhypo_time','simphot_time']
ignoreseg   = ['center_of_flash_x_M','width_of_flash_x_M','matchscore_M',"flashhypo_channel_M"]
ignorelistl = ignorelists + ignoreseg

## Functions

In [3]:
def CheckBorder(tolerance,x,y,z):
    detectorx   =256.35     # In cm
    detectory   =116.5      # Symmetric around 0     
    detectorz   =1036.8
    d=tolerance # border tolerance
    if (0+d) < x < (detectorx-d):
            if (-detectory+d)< y < (detectory-d):
                    if (0+d) < z < (detectorz-d):
                        return True
    return False

In [4]:
# filedict:     Dictionary of root files with pdfcode as key
# ignorelist:   List of columnnames to excluse
# maxn:         Maximum number of rows to import for each file, sampled
# concat:       True, return concatenated dataframe; False, return dict of dataframes with key = filedict key 
# contained:    Drop all events where the true end point is outside of the TPC
# pfp:          Require at least one pfp particle

# - note - Function normalizes the PMT vectors, and replaces the simpphot array with its sum
# - note - Loading the complete dataset with approx 100k events takes a 100MB dataframe

def loadData(filedict,ignorelist,maxn=4300,concat=True,contained=True,pfp=True):
    global labels
    dfdict={}
    for key,fname in filedict.iteritems():
        dftemp=read_root(fname,ignore=ignorelist)
        print 'Dataframe with', labels[key], 'has', len(dftemp.index), 'entries.'
        if contained:
            dftemp=dftemp[dftemp.apply(lambda x: CheckBorder(0,x['true_end_x'],x['true_end_y'],x['true_end_z']), axis=1)]
            print 'Dataframe with', labels[key], 'has', len(dftemp.index), 'contained entries.'
        if pfp:
            dftemp=dftemp[dftemp['nr_pfp']>0]
            print 'Dataframe with', labels[key], 'has', len(dftemp.index), 'entries with at least one PFParticle.'
        if(maxn>0 and maxn<len(dftemp.index)):
            dftemp=dftemp.sample(n=maxn)
        print 'Final dataframe with', labels[key], 'has', len(dftemp.index), 'entries.'    
        dftemp['recphot_channel'] = dftemp['recphot_channel'].apply(lambda x: x/sum(x))
        dftemp['flashhypo_channel'] = dftemp['flashhypo_channel'].apply(lambda x: x/sum(x))
        dftemp['simphot_sum'] = dftemp['simphot_channel'].apply(lambda x: sum(x))
        dftemp['true_time']=dftemp['true_time']/1000
        dftemp.drop(['simphot_channel'], 1,inplace=True)
        dftemp["true_pdg"]=dftemp["true_pdg"].map(label)
        if not "flashhypo_channel_M" in ignorelist:
            dftemp['flashhypo_channel_M'] = dftemp['flashhypo_channel_M'].apply(lambda x: x/sum(x))
        dftemp.fillna(-1,inplace=True)
        dfdict[key]=dftemp
    if concat:
        return pd.concat(dfdict.values(),ignore_index=True,copy=False)
    else:
        return dfdict

In [5]:
def sciNot(x):
    return "{:.2g}".format(x)

In [6]:
def anglevec(v1,v2):
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))  

## Blocks with analysis and plots

In [7]:
df=loadData(filedict,ignorelists,maxn=18200,concat=True,contained=False,pfp=True)
df.info(memory_usage='deep')

Dataframe with Electrons has 20700 entries.
Dataframe with Electrons has 20044 entries with at least one PFParticle.
Final dataframe with Electrons has 18200 entries.
Dataframe with Protons has 22080 entries.
Dataframe with Protons has 18203 entries with at least one PFParticle.
Final dataframe with Protons has 18200 entries.
Dataframe with Muons has 23078 entries.
Dataframe with Muons has 21967 entries with at least one PFParticle.
Final dataframe with Muons has 18200 entries.
Dataframe with Gammas has 25400 entries.
Dataframe with Gammas has 22636 entries with at least one PFParticle.
Final dataframe with Gammas has 18200 entries.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72800 entries, 0 to 72799
Data columns (total 34 columns):
nr_pfp                 72800 non-null uint16
q_z_total              72800 non-null float32
q_z_sps                72800 non-null float32
q_z_hit                72800 non-null float32
flashhypo_channel      72800 non-null object
center_of_charge_x    

In [8]:
df.head(5)

,nr_pfp,q_z_total,q_z_sps,q_z_hit,flashhypo_channel,center_of_charge_x,center_of_charge_y,center_of_charge_z,true_pdg,true_energy,...,center_of_flash_z,width_of_flash_x,width_of_flash_y,width_of_flash_z,matchscore,flashhypo_channel_M,center_of_flash_x_M,width_of_flash_x_M,matchscore_M,simphot_sum
0,5,182471.562500,169381.140625,176362.281250,"[0.13601674323, 0.203860377776, 0.094740954322...",120.235909,41.850025,988.755005,Electron,1.748988,...,933.197876,72.595573,42.664963,79.641228,6.568062,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0.0,0.0,-1.0,4044
1,5,120291.632812,103362.250000,114955.703125,"[3.92094886985e-05, 6.03653173276e-05, 6.47211...",57.239719,-23.183489,383.692444,Electron,1.092242,...,375.941986,33.364498,39.964901,99.546684,8.969843,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0.0,0.0,-1.0,6929
2,3,32005.583984,30350.148438,32005.582031,"[0.00148701841991, 0.000676657638472, 0.001086...",245.852539,-56.391006,467.847351,Electron,1.327636,...,439.503174,192.263168,46.256001,162.245941,3.246179,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0.0,0.0,-1.0,178
3,2,4105.322266,4035.143799,4035.143799,"[0.0188449176219, 0.0294854754917, 0.023829953...",51.657173,110.518448,820.797058,Electron,1.628061,...,836.351074,0.042642,41.060749,119.737694,0.604561,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0.0,0.0,-1.0,697
4,2,8673.943359,4811.913574,4863.978516,"[0.0126409265573, 0.0115747637445, 0.022430632...",117.292114,-70.081612,746.861816,Electron,0.220210,...,719.755859,0.168434,44.501995,113.559685,5.401861,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0.0,0.0,-1.0,658


In [9]:
if 0: # Heatmap of the correlation of the dataframe
    sns.set(font_scale=2.5) 
    fig, ax = plt.subplots(figsize=(20,12.36))
    sns.heatmap(df[df["true_pdg"]==label[11]].drop(ignoreseg+["true_pdg"],1).corr())
    
if 0: # Clustermap of the correlation  
    sns.set(font_scale=2.) 
    cg = sns.clustermap(df[df["true_pdg"]==label[13]].drop(["true_pdg"],1).corr(),figsize=(22,13),metric='euclidean')
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
    #cg.set_title("Clustermap of correlation for "+labels[13])
    #plt.subplots_adjust(bottom=0.15)
    cg.ax_row_dendrogram.set_visible(False)
    cg.ax_col_dendrogram.set_visible(False)
    cg.cax.set_visible(False)
    cg.savefig("clustermap.pdf")

In [10]:
if 0: # Plots that compare the PE spectrum 

    pmt = range(nrPMT)
    plt.style.use('default')
    fig, axarr = plt.subplots(2,2,figsize=(10,7.5))
    
    i=0
    for key in label:

        row=df[(df["true_pdg"]==label[key]) & (df["matchscore"]!=-1)].sample()

        r = int(i/2)
        c = i%2
        i+=1
        print row[['center_of_flash_x','width_of_flash_x']]
        
        dx    = str(int((row['center_of_charge_x']-row['center_of_flash_x']).item()))+"cm"
        dx_M  = str(int((row['center_of_charge_x']-row['center_of_flash_x_M']).item()))+"cm"

        axarr[r][c].set_title(label[key]+" with E= "+str(int(row['true_energy']*100)/100)+"GeV")
        axarr[r][c].set_ylabel("Normalized PE count")
        axarr[r][c].set_xlabel("PMT (OpDet ID)")
        axarr[r][c].plot(pmt,row['recphot_channel'].item(),marker='.', alpha=0.5, label="OpFlash PE spectrum")
        if (key==13 or key==2212):
            axarr[r][c].plot(pmt,row['flashhypo_channel_M'].item(),marker='.',c="#ff7f0e", alpha=0.5, label=("MIP Path: score="+sciNot(row['matchscore_M'].item())+r", $\Delta$x="+dx_M))
        axarr[r][c].plot(pmt,row['flashhypo_channel'].item(),marker='.', alpha=0.5,c='#2ca02c', label="Charge: score="+sciNot(row['matchscore'].item())+r", $\Delta$x="+dx_M)

        axarr[r][c].set_xlim(0,31)
        axarr[r][c].set_ylim(0,0.4)
        axarr[r][c].legend()
        axarr[r][c].grid(alpha=0.5)

    plt.tight_layout()
    plt.savefig("spectrum.pdf")


In [11]:
if 0: # Histogram of the matchscores for different particle types
    plt.style.use('default')
    group= df[(df['nr_pfp']>0) & (df['matchscore']!=-1)][['true_pdg','matchscore']].groupby('true_pdg')

    fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10*.8, 6.18*.8))
    axes[0].set_xlabel('Matchscore')
    axes[0].set_xlim(-1.5,20)
    axes[1].set_xlabel(r'$\log$(Matchscore)')
    axes[0].set_ylabel('Events per bin')
    
    for key in label:
        totalrows=len(df[df["true_pdg"]==label[key]])
        dftemp= group.get_group(label[key])['matchscore'].values
        dftemplog = np.log(dftemp)
        axes[0].hist(dftemp,bins=int(round(totalrows/100)),histtype='step',label=label[key]+" ("+str(int(round(len(dftemp)/totalrows,2)*100))+"%)")
        axes[1].hist(dftemplog,bins=int(round(totalrows/100)),histtype='step',label=label[key])
    axes[0].legend(loc=0)
    axes[0].grid(alpha=0.5)
    axes[1].grid(alpha=0.5)
    plt.tight_layout()
    fig.subplots_adjust(top=0.92)
    fig.suptitle("Matchscore for particles with at least one PFP and OpFlash in event (selection percentage)")
    
    plt.savefig("scoredist.pdf")
    
    
if 0: # Histogram of the matchscores for tracklike, compared with lightpath
    plt.style.use('default')
    group= df[(df['nr_pfp']>0) & (df['matchscore']!=-1)][['true_pdg','matchscore']].groupby('true_pdg')
    group_M= df[(df['nr_pfp']>0) & (df['matchscore_M']!=-1)][['true_pdg','matchscore_M']].groupby('true_pdg')

    fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10*.8, 6.18*.8))
    axes[0].set_xlabel('Matchscore')
    axes[0].set_xlim(-1.5,20)
    axes[1].set_xlabel(r'$\log$(Matchscore)')
    axes[0].set_ylabel('Events per bin')
    
    for key in [13,2212]:
        totalrows=len(df[df["true_pdg"]==label[key]])
        dftemp= group.get_group(label[key])['matchscore'].values
        dftemp_M= group_M.get_group(label[key])['matchscore_M'].values
        dftemplog = np.log(dftemp)
        axes[0].hist(dftemp,bins=int(round(totalrows/100)),histtype='step',label=label[key]+" ("+str(int(round(len(dftemp)/totalrows,2)*100))+"%) - Charge")
        axes[1].hist(dftemplog,bins=int(round(totalrows/100)),histtype='step',label=label[key])
        axes[0].hist(dftemp_M,bins=int(round(totalrows/100)),histtype='step',label=label[key]+" ("+str(int(round(len(dftemp_M)/totalrows,2)*100))+"%) - MIP Path")
        axes[1].hist( np.log(dftemp_M),bins=int(round(totalrows/100)),histtype='step',label=label[key])
        
    axes[0].legend(loc=0)
    axes[0].grid(alpha=0.5)
    axes[1].grid(alpha=0.5)
    plt.tight_layout()
    fig.subplots_adjust(top=0.92)
    fig.suptitle("Matchscore for particles with at least one PFP and OpFlash in event (selection percentage)")
    
    plt.savefig("scoredist.pdf")
    

In [12]:
if 0: # Function to plot the angle between the center of charge and the momentum direction. This will only work if the number of different particles is equal!
    dfplus = df[(df['nr_pfp']>0)]
    angle=[]
    #This loop is inefficient, could be improved by an apply function on the dataframe directly
    for index, row in dfplus.iterrows():
        dir_sp = np.array([ (row['center_of_charge_x']-row['true_x']) , (row['center_of_charge_y']-row['true_y']), (row['center_of_charge_z']-row['true_z'])])
        dir_p  = np.array([  row['true_px'], row['true_py'], row['true_pz'] ])
        angle.append(anglevec(dir_sp,dir_p))
    
    plt.style.use('default')
    print len(dfplus["true_pdg"].values)
    print len(angle)
    dfangle = pd.DataFrame({'Particle type':dfplus["true_pdg"].values,'Angle':angle})
    #dfangle.set_index('PDG',inplace=True)
    p = dfangle.pivot(columns='Particle type', values='Angle')

    ax =p.plot(kind='hist', by='PDG',stacked=True, bins=100,title="Angle between center of charge and true direction")
    ax.set_xlabel('Angle')
    ax.set_xlim((0, np.pi))
    ax.set_xticks([0, np.pi/4,np.pi/2, 3*np.pi/4,np.pi])
    ax.set_xticklabels(['0', '$\pi/4$','$\pi/2$','$3\pi/4$', '$\pi$'])
    ax.grid(alpha=0.5)
    plt.tight_layout()
    plt.savefig("centerchargeangle.pdf")

In [50]:
if 0: # Joint Plots to compare the flash location with the true interaction location and the center of charge. IMPORTANT PFP>0 or this does not make sense
    sns.set(font_scale=1.5) 
    plt.style.use('default')
    sns.set_palette("GnBu_d")
    #sns.set_palette(sns.cubehelix_palette(3,light=.01,dark=.1))
    
    df["center_of_flash_y - center_of_charge_y [cm]"]=df["center_of_flash_y"]-df["center_of_charge_y"]
    df["center_of_flash_z - center_of_charge_z [cm]"]=df["center_of_flash_z"]-df["center_of_charge_z"]
    g = sns.jointplot(x='center_of_flash_y - center_of_charge_y [cm]',y='center_of_flash_z - center_of_charge_z [cm]',bins='log',data=df,size=7,kind='hex',stat_func=None,gridsize=20,marginal_kws=dict(hist=False, kde=True,kde_kws={"shade": True}))
    cax = g.fig.add_axes([.78, .63, .01, .2])
    cbar=plt.colorbar(cax=cax,ticks=[1,2,3])
    cbar.ax.set_yticklabels([r'$10$', '$10^2$', '$10^3$'])
    plt.savefig('hexbinchrge.pdf')
    
    df["center_of_flash_y - true_y [cm]"]=df["center_of_flash_y"]-df["true_y"]
    df["center_of_flash_z - true_z [cm]"]=df["center_of_flash_z"]-df["true_z"]
    g=sns.jointplot(x='center_of_flash_y - true_y [cm]',y='center_of_flash_z - true_z [cm]',bins='log',data=df,size=7,kind='hex',stat_func=None,gridsize=20,marginal_kws=dict(hist=False, kde=True,kde_kws={"shade": True}))
    cax = g.fig.add_axes([.78, .63, .01, .2])
    cbar=plt.colorbar(cax=cax,ticks=[1,2,3])
    cbar.ax.set_yticklabels([r'$10$', '$10^2$', '$10^3$'])
    plt.savefig('hexbintrue.pdf')
    
    df["true_y - center_of_charge_y [cm]"]=df["true_y"]-df["center_of_charge_y"]
    df["true_z - center_of_charge_z [cm]"]=df["true_z"]-df["center_of_charge_z"]
    g=sns.jointplot(x='true_y - center_of_charge_y [cm]',y='true_z - center_of_charge_z [cm]',bins='log',data=df,size=7,kind='hex',stat_func=None,gridsize=20,marginal_kws=dict(hist=False, kde=True,kde_kws={"shade": True}))
    cax = g.fig.add_axes([.78, .63, .01, .2])
    cbar=plt.colorbar(cax=cax,ticks=[1,2,3])
    cbar.ax.set_yticklabels([r'$10$', '$10^2$', '$10^3$'])
    plt.savefig('hexbinchrgetrue.pdf')

In [14]:
if 0: # Histograms in X,Y,Z comparing center_of_flash location with center_of_charge and true position. Hue for different proton/Muon LightPath/LightCharge
    plt.style.use('default')
    group= df[(df['nr_pfp']>0) & (df['matchscore']>-.1) ][['true_pdg','true_x','true_y','true_z','center_of_charge_x','center_of_charge_y','center_of_charge_z','center_of_flash_x','center_of_flash_y','center_of_flash_z']].groupby('true_pdg')
    #& 
    fig, axes = plt.subplots(nrows=1, ncols=3,figsize=(10, 4),sharey=False)
    axes[0].set_xlabel('x [cm]')
    axes[1].set_xlabel('y [cm]')
    axes[2].set_xlabel('z [cm]')
    axes[0].set_ylabel('Normalized')
    axes[1].set_ylabel('Normalized')
    axes[2].set_ylabel('Normalized')
    
    for key in label:
        totalrows=len(df[df["true_pdg"]==label[key]])
        dxtc = (group.get_group(label[key])['true_x']-group.get_group(label[key])['center_of_charge_x']).values
        dytc = (group.get_group(label[key])['true_y']-group.get_group(label[key])['center_of_charge_y']).values
        dztc = (group.get_group(label[key])['true_z']-group.get_group(label[key])['center_of_charge_z']).values

        axes[0].hist(dxtc,bins=int(round(totalrows/200)),histtype='step',normed=True,label=label[key]+" ("+str(int(round(len(dxtc)/totalrows,2)*100))+"%)")
        axes[1].hist(dytc,bins=int(round(totalrows/200)),histtype='step',normed=True,label=label[key]+" ("+str(int(round(len(dytc)/totalrows,2)*100))+"%)")
        axes[2].hist(dztc,bins=int(round(totalrows/200)),histtype='step',normed=True,label=label[key]+" ("+str(int(round(len(dztc)/totalrows,2)*100))+"%)")
        
        
        
    axes[2].legend(bbox_to_anchor=(1.1, .7))
    axes[0].grid(alpha=0.5)
    axes[1].grid(alpha=0.5)
    axes[2].grid(alpha=0.5)
    axes[0].set_xlim(-200,200)
    axes[1].set_xlim(-125,125)
    axes[2].set_xlim(-200,200)
    #axes[2].set_ylim(0,0.003)
    #plt.yscale('log', nonposy='clip')
    
    plt.tight_layout()
    fig.subplots_adjust(right=0.8)
    fig.subplots_adjust(top=0.92)
    fig.suptitle("Difference between center of charge and true interaction position")
    
    plt.savefig("flashtrue.pdf")


if 0: # Histograms in X,Y,Z comparing center_of_flash location with center_of_charge and true position. Hue for different particles
    plt.style.use('default')
    group= df[(df['nr_pfp']>0) & (df['matchscore']!=-1) ][['true_pdg','true_x','true_y','true_z','center_of_charge_x','center_of_charge_y','center_of_charge_z','center_of_flash_x','center_of_flash_y','center_of_flash_z']].groupby('true_pdg')
    
    fig, axes = plt.subplots(nrows=1, ncols=3,figsize=(10, 4),sharey=False)
    axes[0].set_xlabel('x [cm]')
    axes[1].set_xlabel('y [cm]')
    axes[2].set_xlabel('z [cm]')
    axes[0].set_ylabel('Normalized')
    axes[1].set_ylabel('Normalized')
    axes[2].set_ylabel('Normalized')
    
    for key in label:
        totalrows=len(df[df["true_pdg"]==label[key]])
        dxfc = (group.get_group(label[key])['center_of_flash_x']-group.get_group(label[key])['center_of_charge_x']).values
        dyfc = (group.get_group(label[key])['center_of_flash_y']-group.get_group(label[key])['center_of_charge_y']).values
        dzfc = (group.get_group(label[key])['center_of_flash_z']-group.get_group(label[key])['center_of_charge_z']).values

        axes[0].hist(dxfc,bins=int(round(totalrows/200)),histtype='step',normed=True,label=label[key]+" ("+str(int(round(len(dxfc)/totalrows,2)*100))+"%)")
        axes[1].hist(dyfc,bins=int(round(totalrows/200)),histtype='step',normed=True,label=label[key]+" ("+str(int(round(len(dyfc)/totalrows,2)*100))+"%)")
        axes[2].hist(dzfc,bins=int(round(totalrows/100)),histtype='step',normed=True,label=label[key]+" ("+str(int(round(len(dzfc)/totalrows,2)*100))+"%)")
        
        
      
    axes[2].legend(bbox_to_anchor=(1.1, .69))
    axes[0].grid(alpha=0.5)
    axes[1].grid(alpha=0.5)
    axes[2].grid(alpha=0.5)
    axes[2].set_xlim(-200,200)
    axes[1].set_xlim(-125,125)
    axes[0].set_xlim(-200,200)
    #axes[2].set_xlim(-1200,1200)
    #plt.yscale('log', nonposy='clip')
    
    plt.tight_layout()
    fig.subplots_adjust(right=0.8)
    fig.subplots_adjust(top=0.92)
    fig.suptitle("Difference between center of charge and center of flash")
    
    plt.savefig("flashcharge.pdf")
    

## Sandbox beyond this point

In [ ]:
if 0: # Block to check how much events survive certain selections
    print df.shape
    print df.loc[(df['matchscore']==-1)].shape
    print df.loc[(df['matchscore']<0.0000000001) & (df['matchscore']>(-1.0))].shape

In [ ]:
if 0: # Comparison of the flashhypothesis and the opFlash for specific score range
    sns.set(font_scale=1)
    for i in range(5):
        n = index=np.random.randint(50)
        data=df.loc[(df['matchscore']<0.001) & (df['matchscore']>(-1.0))].iloc[n]

        print(data.shape)
        y1=data['flashhypo_channel']
        y2=data['recphot_channel']
        score =data['matchscore']

        x=range(0,32)
        plt.plot(x,y1)
        plt.plot(x,y2)
        #plt.plot(x,y3)
        plt.title('score:' + sciNot(score))
        plt.show()

In [ ]:
 if 0: #Lineair regression gogogo #failed
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import LogisticRegression
    from sklearn import metrics
    
    dflearn11 = df[(df['nr_pfp']>0) & (df['matchscore']>0.01) & (df['true_pdg']==label[22]) & (df['simphot_sum']>1000) & (df['simphot_sum']<15000) ].copy()
    
    dflearn11['center_of_charge_y_lin']=dflearn11['center_of_charge_y'].apply(lambda x:abs(x))
    dflearn11['center_of_charge_z_lin']=dflearn11['center_of_charge_z'].apply(lambda x: x if x < 1036.8/2 else 1036.8-x )
    
    
    X11 = dflearn11[['center_of_charge_x', 'center_of_charge_y_lin', 'center_of_charge_z_lin','q_z_total']]
    y11 = dflearn11['simphot_sum']
    
    X11_train, X11_test, y11_train, y11_test = train_test_split(X11, y11, test_size=0.8, random_state=101)
    
    lm = LinearRegression()
    lm.fit(X11_train,y11_train)
    
    #coeff_df = pd.DataFrame(lm.coef_,X11.columns,columns=['Coefficient'])
    
    predictions = lm.predict(X11_test)
    
    #dataframe = pd.DataFrame([y_test,predictions],range(len(predictions)),columns=['Test','Prediction'])
    #dataframe.head()
    sns.regplot(y11_test,predictions)
    
    dflearn['center_of_charge_x'].head(50)
    coeff_df